# Object Detection

In [1]:
!nvidia-smi

Wed Apr  3 23:56:37 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.89.02    Driver Version: 525.89.02    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:3B:00.0 Off |                    0 |
| N/A   30C    P0    25W / 250W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:D8:00.0 Off |                    0 |
| N/A   

In [5]:
from zipfile import ZipFile

# Path to your ZIP file
zip_file_path = '/user/charviku/DL_Project/final_data.zip'
# Destination directory where you want to extract the files
extraction_directory = '/user/charviku/DL_Project/in'

# Extract the ZIP file
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_directory)

print("Extraction completed.")


Extraction completed.


In [8]:
import xml.etree.ElementTree as ET
import os
from PIL import Image

# Load class names and create a mapping to IDs
def load_class_names(class_file_path):
    with open(class_file_path, 'r') as file:
        class_names = file.read().strip().split('\n')
    return {name: i for i, name in enumerate(class_names)}

def convert_voc_to_yolo(voc_xml_file, class_mapping, img_width, img_height):
    tree = ET.parse(voc_xml_file)
    root = tree.getroot()
    yolo_format = []

    for member in root.findall('object'):
        classname = member.find('name').text
        class_id = class_mapping[classname]

        bndbox = member.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        x_center = ((xmin + xmax) / 2) / img_width
        y_center = ((ymin + ymax) / 2) / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height

        yolo_format.append(f"{class_id} {x_center} {y_center} {width} {height}")

    return yolo_format

def process_dataset(dataset_directory, class_file_path):
    class_mapping = load_class_names(class_file_path)

    for class_dir in os.listdir(dataset_directory):
        class_path = os.path.join(dataset_directory, class_dir)
        if os.path.isdir(class_path):
            for file in os.listdir(class_path):
                if file.endswith('.xml'):
                    img_file = os.path.splitext(file)[0] + '.jpg'
                    img_path = os.path.join(class_path, img_file)
                    xml_path = os.path.join(class_path, file)

                    # Use PIL to get image dimensions
                    with Image.open(img_path) as img:
                        img_width, img_height = img.size

                    yolo_annotations = convert_voc_to_yolo(xml_path, class_mapping, img_width, img_height)
                    yolo_annotation_text = "\n".join(yolo_annotations)

                    # Save YOLO annotations to a .txt file
                    txt_filename = os.path.splitext(xml_path)[0] + '.txt'
                    with open(txt_filename, 'w') as f:
                        f.write(yolo_annotation_text)

# Assuming your class file path and dataset directory are as follows:
class_file_path = '/user/charviku/DL_Project/Final_classes.txt'
dataset_directory = '/user/charviku/DL_Project/in/initial_data_annotated'
process_dataset(dataset_directory, class_file_path)





In [4]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 94.0 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 10.6 MB/s eta 0:00:00
DEPRECATION: matlabengineforpython R2021b has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of matlabengineforpython or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import random
from sklearn.model_selection import train_test_split

# Define your dataset directory and output files
dataset_dir = '/user/charviku/DL_Project/in/initial_data_annotated'
output_train = '/user/charviku/DL_Project/train.txt'
output_val = '/user/charviku/DL_Project/valid.txt'
output_test = '/user/charviku/DL_Project/test.txt'

# Specify the split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1  # Ensures train + val + test = 1.0

# Collect all image file paths
image_paths = []
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.endswith('.jpg'):
            image_paths.append(os.path.join(root, file))

# Split the data
train_val_paths, test_paths = train_test_split(image_paths, test_size=test_ratio, random_state=42)
train_paths, val_paths = train_test_split(train_val_paths, test_size=val_ratio/(train_ratio+val_ratio), random_state=42)

# Function to write paths to a file
def write_paths(file_paths, output_file):
    with open(output_file, 'w') as f:
        for path in file_paths:
            f.write(path + '\n')

# Write the splits to their respective files
write_paths(train_paths, output_train)
write_paths(val_paths, output_val)
write_paths(test_paths, output_test)

print(f"Training images: {len(train_paths)}")
print(f"Validation images: {len(val_paths)}")
print(f"Test images: {len(test_paths)}")


Training images: 3988
Validation images: 499
Test images: 499


In [6]:
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7


Cloning into 'yolov7'...
remote: Enumerating objects: 1197, done.
remote: Total 1197 (delta 0), reused 0 (delta 0), pack-reused 1197
Receiving objects: 100% (1197/1197), 74.23 MiB | 61.69 MiB/s, done.
Resolving deltas: 100% (519/519), done.
Updating files: 100% (108/108), done.
/user/charviku/DL_Project/yolov7


In [7]:
!pip install -r requirements.txt


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.1 MB/s eta 0:00:00a 0:00:01


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 51.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 6.9 MB/s eta 0:00:00ta 0:00:01
DEPRECATION: matlabengineforpython R2021b has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of matlabengineforpython or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [9]:
yaml_content = """
train: /user/charviku/DL_Project/train.txt
val: /user/charviku/DL_Project/valid.txt
test: /user/charviku/DL_Project/test.txt

nc: 100
names: [
  'all_purpose_flour', 'almonds', 'apple', 'apricot', 'asparagus', 'avocado', 'bacon', 'banana', 'barley', 'basil',
  'basmati_rice', 'beans', 'beef', 'beets', 'bell_pepper', 'berries', 'biscuits', 'blackberries', 'black_pepper',
  'blueberries', 'bread', 'bread_crumbs', 'bread_flour', 'broccoli', 'brownie_mix', 'brown_rice', 'butter', 'cabbage',
  'cake', 'cardamom', 'carrot', 'cashews', 'cauliflower', 'celery', 'cereal', 'cheese', 'cherries', 'chicken',
  'chickpeas', 'chocolate', 'chocolate_chips', 'chocolate_syrup', 'cilantro', 'cinnamon', 'clove', 'cocoa_powder',
  'coconut', 'cookies', 'corn', 'cucumber', 'dates', 'eggplant', 'eggs', 'fish', 'garlic', 'ginger', 'grapes', 'honey',
  'jalapeno', 'kidney_beans', 'lemon', 'mango', 'marshmallows', 'milk', 'mint', 'muffins', 'mushroom', 'noodles',
  'nuts', 'oats', 'okra', 'olive', 'onion', 'orange', 'oreo_cookies', 'pasta', 'pear', 'pepper', 'pineapple',
  'pistachios', 'pork', 'potato', 'pumpkin', 'radishes', 'raisins', 'red_chilies', 'rice', 'rosemary', 'salmon', 'salt',
  'shrimp', 'spinach', 'strawberries', 'sugar', 'sweet_potato', 'tomato', 'vanilla_ice_cream', 'walnuts', 'watermelon',
  'yogurt'
]
"""

with open('/user/charviku/DL_Project/ingredients.yaml', 'w') as file:
    file.write(yaml_content.strip())


In [11]:
%cd /user/charviku/DL_Project/yolov7


/user/charviku/DL_Project/yolov7


In [13]:
!python train.py --batch-size 32 --img 320 320 --data /user/charviku/DL_Project/ingredients.yaml --cfg cfg/training/yolov7.yaml --weights 'yolov7.pt' --device 0 --epochs 100 --workers 4

YOLOR 🚀 v0.1-128-ga207844 torch 1.13.1 CUDA:0 (Tesla V100-PCIE-16GB, 16150.875MB)

Namespace(weights='yolov7.pt', cfg='cfg/training/yolov7.yaml', data='/user/charviku/DL_Project/ingredients.yaml', hyp='data/hyp.scratch.p5.yaml', epochs=100, batch_size=32, img_size=[320, 320], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=4, project='runs/train', entity=None, name='exp', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, global_rank=-1, save_dir='runs/train/exp2', total_batch_size=32)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.

 92                -1  1    590336  models.common.Conv                      [256, 256, 3, 2]              
 93      [-1, -3, 51]  1         0  models.common.Concat                    [1]                           
 94                -1  1    525312  models.common.Conv                      [1024, 512, 1, 1]             
 95                -2  1    525312  models.common.Conv                      [1024, 512, 1, 1]             
 96                -1  1   1180160  models.common.Conv                      [512, 256, 3, 1]              
 97                -1  1    590336  models.common.Conv                      [256, 256, 3, 1]              
 98                -1  1    590336  models.common.Conv                      [256, 256, 3, 1]              
 99                -1  1    590336  models.common.Conv                      [256, 256, 3, 1]              
100[-1, -2, -3, -4, -5, -6]  1         0  models.common.Concat                    [1]                           
101                -1  1   1049

     16/99     6.44G   0.02481  0.008267    0.0483   0.08138        54       320
               Class      Images      Labels           P           R      mAP@.5
                 all         499         367       0.292       0.261       0.148       0.119

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     17/99     6.44G   0.02521  0.008244   0.04759   0.08104        38       320
               Class      Images      Labels           P           R      mAP@.5
                 all         499         367       0.211       0.303       0.146       0.119

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     18/99     6.44G   0.02498  0.007996   0.04597   0.07895        48       320
               Class      Images      Labels           P           R      mAP@.5
                 all         499         367       0.309        0.29       0.165       0.135

     Epoch   gpu_mem       box       obj       cls     total    labels

               Class      Images      Labels           P           R      mAP@.5
                 all         499         367       0.362       0.441       0.394       0.341

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     41/99     6.44G    0.0222  0.007117   0.02663   0.05595        37       320
               Class      Images      Labels           P           R      mAP@.5
                 all         499         367       0.535       0.405       0.421       0.361

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     42/99     6.44G   0.02216  0.007183   0.02588   0.05522        33       320
               Class      Images      Labels           P           R      mAP@.5
                 all         499         367       0.447       0.417       0.409       0.356

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     43/99     6.44G   0.02219  0.007101   0.02547   0.05476        47


     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     65/99     6.44G   0.01821  0.006159   0.01588   0.04025        40       320
               Class      Images      Labels           P           R      mAP@.5
                 all         499         367       0.534       0.474       0.501       0.452

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     66/99     6.44G   0.01829  0.006155   0.01586   0.04031        43       320
               Class      Images      Labels           P           R      mAP@.5
                 all         499         367       0.442       0.504       0.508       0.459

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     67/99     6.44G   0.01856  0.006116    0.0157   0.04038        38       320
               Class      Images      Labels           P           R      mAP@.5
                 all         499         367       0.418       0.515       0.502  

     89/99     6.44G   0.01505  0.005278   0.01022   0.03055        48       320
               Class      Images      Labels           P           R      mAP@.5
                 all         499         367       0.476       0.551       0.547       0.502

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     90/99     6.44G   0.01501  0.005181   0.01009   0.03027        55       320
               Class      Images      Labels           P           R      mAP@.5
                 all         499         367       0.494       0.518       0.537       0.496

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     91/99     6.44G   0.01523  0.005123   0.01002   0.03037        62       320
               Class      Images      Labels           P           R      mAP@.5
                 all         499         367       0.468       0.533       0.528       0.484

     Epoch   gpu_mem       box       obj       cls     total    labels